In [ ]:
#THE FOLLOWING CODE SCANS THE DATASET AND COUNTS THE AMOUNT OF FILE PRESENT#

import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi 
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

#SAMPLE RATE FOR AUDIO PLAYBACK#
samplerate = 16000

#CHECK THE FOLDER, RETURN ERROR IF NOT FOUND#
data_dir = pathlib.Path('Project Dataset')
if not data_dir.exists():
  print("Dataset Not Found")
else:
  print("Dataset", data_dir ,"Found!")

#LOOK FOR MIDI FILES, COUNT THE TOTAL AMOUNT OF MIDI FILES#
filenames = glob.glob(str(data_dir/'*.mid*'))
print('Number of files:', len(filenames))

In [ ]:
#PULL A FILE FROM THE DATA SET AND PLAY IT, LABEL THE INSTRUMENT USED AND THE NOTES PLAYED#

sample_file = filenames[23]
print("File Sampled:",sample_file)
pm = pretty_midi.PrettyMIDI(sample_file) #LOAD THE FILE INTO A PRETTY MIDI OBJECT#
endtime = round(pm.get_end_time(), 0)
length = int(endtime) #FETCH FILE LENGTH FOR PARSING LATER, MUST BE AN INTEGER TO AVOID KERNEL ERRORS
print("File Is",endtime,"Seconds")
#DEFINE A SUBROUTINE TO DISPLAY THE AUDIO FILE#
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=length): #PROGRAM KNOWS HOW LONG TO SET THE FILE
  waveform = pm.fluidsynth(fs=samplerate)
  waveform_short = waveform[:seconds*samplerate]
  return display.Audio(waveform_short, rate=samplerate)

#FIND THE INSTRUMENTS, COUNT AND DEFINE THEM#
print("Number Of Instruments:", len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print("Instrument Used:", instrument_name)

#ESTIMATE THE TEMPO OF THE MUSIC#
tempo = pm.estimate_tempo()
tempo_rounded = round(tempo, 0)
print("File Tempo:", tempo_rounded,"Bpm")

#FIND THE NOTES USED IN THE MUSIC#
for i, note in enumerate(instrument.notes[:10]):
  note_name = pretty_midi.note_number_to_name(note.pitch)
  duration = note.end - note.start
  print(f'{i}: pitch={note.pitch}, note_name={note_name},'
        f' duration={duration:.4f}')

#CONVERT THE MIDI TO NOTES THAT ARE PLAYABLE#
listNotes = np.zeros(1)
def midi_to_notes(midi_file: str) -> pd.DataFrame:
    pm = pretty_midi.PrettyMIDI(midi_file)
    instrument = pm.instruments[0]
    notes = collections.defaultdict(list)

#SORT THE NOTES INTO START TIME ORDER#
    sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start
    for note in sorted_notes:
        start = note.start
        end = note.end
        notes['pitch'].append(note.pitch)
        notes['start'].append(start)
        notes['end'].append(end)
        notes['step'].append(start - prev_start)
        notes['duration'].append(end - start)
        prev_start = start
    return pd.DataFrame({name: np.array(value) for name, value in notes.items()})
raw_notes = midi_to_notes(sample_file)
array = np.zeros(1)
dataArray = midi_to_notes(sample_file)
notename = np.zeros(1)
df = pd.DataFrame({'Note':(dataArray.pitch)}, index=[0])
dataArray.pitch.to_csv('Raw Data.csv', index=True)
np.savetxt('pitch.csv',dataArray.pitch)
raw_notes.head()
display_audio(pm)

In [ ]:
#PLOT GRAPHS#

def plot_piano_roll(notes: pd.DataFrame, count: Optional[int] = None):
  if count:
    title = f'First {count} notes'
  else:
    title = f'Whole track'
    count = len(notes['pitch'])
  plt.figure(figsize=(20, 4))
  plot_pitch = np.stack([notes['pitch'], notes['pitch']], axis=0)
  plot_start_stop = np.stack([notes['start'], notes['end']], axis=0)
  plt.plot(
      plot_start_stop[:, :count], plot_pitch[:, :count], color="b", marker=".")
  plt.xlabel('Time [s]')
  plt.ylabel('Pitch')
  _ = plt.title(title)

def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pm.instruments[0]
  notes = collections.defaultdict(list)

  # Sort the notes by start time
  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    notes['notes'].append(notes)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})
plot_piano_roll(raw_notes, count=200)
plot_piano_roll(raw_notes)

def plot_distributions(notes: pd.DataFrame, drop_percentile=2.5):
  plt.figure(figsize=[15, 5])
  plt.subplot(1, 3, 1)
  sns.histplot(notes, x="pitch", bins=20)

  plt.subplot(1, 3, 2)
  max_step = np.percentile(notes['step'], 100 - drop_percentile)
  sns.histplot(notes, x="step", bins=np.linspace(0, max_step, 21))

  plt.subplot(1, 3, 3)
  max_duration = np.percentile(notes['duration'], 100 - drop_percentile)
  sns.histplot(notes, x="duration", bins=np.linspace(0, max_duration, 21))
plot_distributions(raw_notes)

In [ ]:
#TRAINING THE FILE USING SKLEARN#
df2 = pd.read_csv("Raw Data.csv")
df2.head()
X = df2.drop('pitch', axis = 1)
y = df2['pitch']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)
rfc = RandomForestClassifier(n_estimators = 200)
rfc.fit(X_train, y_train)
predictionsRFC = rfc.predict(X_test)
print(predictionsRFC)
confMatRFC = confusion_matrix(y_test, predictionsRFC, labels = rfc.classes_) 
print(" ")
print("Confusion matrix: \n",confMatRFC)
print(" ")
print("Plot of confusion matrix:")
disp = ConfusionMatrixDisplay(confusion_matrix = confMatRFC, display_labels = rfc.classes_)
np.savetxt('Predicted Pitch.csv',predictionsRFC)
disp.plot()
dir(disp)
#LSTM LIKELY A BETTER OPTION, MANY TREES DOESN'T PREDICT WELL#

In [ ]:
#LTSM ATTEMPT TO EXTRACT PITCH AND DURATION#
sample_file2 = filenames[23]
print("File Sampled:",sample_file2)
pm = pretty_midi.PrettyMIDI(sample_file2) #LOAD THE FILE INTO A PRETTY MIDI OBJECT#
endtime = round(pm.get_end_time(), 0)
length = int(endtime) #FETCH FILE LENGTH FOR PARSING LATER, MUST BE AN INTEGER TO AVOID KERNEL ERRORS
print("File Is",endtime,"Seconds")
#DEFINE A SUBROUTINE TO DISPLAY THE AUDIO FILE#
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=length): #PROGRAM KNOWS HOW LONG TO SET THE FILE
  waveform = pm.fluidsynth(fs=samplerate)
  waveform_short = waveform[:seconds*samplerate]
  return display.Audio(waveform_short, rate=samplerate)

#FIND THE INSTRUMENTS, COUNT AND DEFINE THEM#
print("Number Of Instruments:", len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print("Instrument Used:", instrument_name)

#ESTIMATE THE TEMPO OF THE MUSIC#
tempo = pm.estimate_tempo()
tempo_rounded = round(tempo, 0)
print("File Tempo:", tempo_rounded,"Bpm")

#CONVERT THE MIDI TO NOTES THAT ARE PLAYABLE#
listNotes = np.zeros(1)
def midi_to_notes(midi_file: str) -> pd.DataFrame:
    pm = pretty_midi.PrettyMIDI(midi_file)
    instrument = pm.instruments[0]
    notes = collections.defaultdict(list)

#SORT THE NOTES INTO START TIME ORDER#
    sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start
    for note in sorted_notes:
        start = note.start
        end = note.end
        notes['pitch'].append(note.pitch)
        notes['start'].append(start)
        notes['end'].append(end)
        notes['step'].append(start - prev_start)
        notes['duration'].append(end - start)
        prev_start = start
    return pd.DataFrame({name: np.array(value) for name, value in notes.items()})
raw_notes = midi_to_notes(sample_file2)
array = np.zeros(1)
dataArray = midi_to_notes(sample_file2)
notename = np.zeros(1)
df = pd.DataFrame({'Note':(dataArray.pitch)}, {'Duration':(dataArray.duration)})
dataArray.pitch.to_csv('Raw Data.csv', index=True)
np.savetxt('ltsm.csv', np.c_[dataArray.pitch, dataArray.duration, dataArray.start, dataArray.end, dataArray.step], delimiter=',', header = 'Pitch,Duration,Start,End,Step', comments='', fmt='%1.2f')
#PROGRAM HAS NOW SAVED DATA OF THE FILE TO A CSV#
print("Data Extracted Successfully!")

In [ ]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

np.savetxt('ltsm (predicted).csv', np.c_[testPredict], delimiter=',', header = 'Predicted Duration', comments='', fmt='%1.2f')
# SAVE np.c_[testPredict] AS DIFFERENT VARIABLE EACH TIME#

In [ ]:
#ATTEMPT TO TRAIN OTHER ASPECTS IN ORDER TO CREATE A FULL SONG#

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm.csv', usecols=[0], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict2 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict2 = scaler.inverse_transform(testPredict2)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict2[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict2

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()


In [ ]:

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm.csv', usecols=[2], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70) #THIS NEEDS CHANGING, IT'S ONLY TRAINING THE LAST 30% OF START#
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict3 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict3 = scaler.inverse_transform(testPredict3)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict3[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict3

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
preDur = np.c_[testPredict]
prePit = np.c_[testPredict2]
preSta = np.c_[testPredict3]
np.savetxt('ltsm (predicted).csv',np.hstack([preDur,prePit,preSta]), delimiter=',', header = 'Predicted Duration, Predicted Pitch, Predicted Start', comments='', fmt='%1.2f')

In [ ]:

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm.csv', usecols=[3], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70) #THIS NEEDS CHANGING, IT'S ONLY TRAINING THE LAST 30% OF START#
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict4 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict4 = scaler.inverse_transform(testPredict4)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict4[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict4

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
preDur = np.c_[testPredict]
prePit = np.c_[testPredict2]
preSta = np.c_[testPredict3]
preEnd = np.c_[testPredict4]
np.savetxt('ltsm (predicted).csv',np.hstack([preDur,prePit,preSta,preEnd]), delimiter=',', header = 'Predicted Duration, Predicted Pitch, Predicted Start, Predicted End', comments='', fmt='%1.2f')

In [ ]:

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm.csv', usecols=[4], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70) #THIS NEEDS CHANGING, IT'S ONLY TRAINING THE LAST 30% OF START#
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict5 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict5 = scaler.inverse_transform(testPredict5)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict5[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict5

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
preDur = np.c_[testPredict]
prePit = np.c_[testPredict2]
preSta = np.c_[testPredict3]
preEnd = np.c_[testPredict4]
preSte = np.c_[testPredict5]
np.savetxt('ltsm (predicted).csv',np.hstack([preDur,prePit,preSta,preEnd,preSte]), delimiter=',', header = 'Predicted Duration, Predicted Pitch, Predicted Start, Predicted End, Predicted Step', comments='', fmt='%1.2f')

In [ ]:
np.savetxt('ltsm (predicted).csv',np.hstack([preDur,prePit,(preSta-preSta[0]),(preEnd-preEnd[0]),preSte]), delimiter=',', header = 'duration,pitch,start,end,step', comments='', fmt='%1.2f')

In [ ]:
DataFrame = pandas.read_csv('ltsm (predicted).csv', engine='python')
def notes_to_midi(
  notes: DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm
print(DataFrame)
example_file = 'example.midi'
example_pm = notes_to_midi(
    DataFrame, out_file=example_file, instrument_name=instrument_name)
PrettyMIDI(midi_file = 'example.midi', resolution = 220, initial_tempo = 80)
display_audio(example_pm)

In [ ]:
plot_piano_roll(DataFrame, count=200)
plot_piano_roll(DataFrame)
plot_distributions(DataFrame)

In [ ]:
#WARNING THIS TAKES A LONG TIME#
num_files = len(filenames)
all_notes = []
for f in filenames[:num_files]:
  notes = midi_to_notes(f)
  all_notes.append(notes)

all_notes = pd.concat(all_notes)
print(all_notes)
np.savetxt('ltsm(completed).csv', all_notes, delimiter=',', header = 'Pitch,Duration,Start,End,Step', comments='', fmt='%1.2f')
print("All Data Extracted Successfully!")


In [ ]:

#WARNING THIS CELL TAKES A VERY LONG TIME TO RUN! ONLY RUN THIS ON A POWERFUL PC#


import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm(completed).csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm(completed).csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm(completed).csv', usecols=[0], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict2 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict2 = scaler.inverse_transform(testPredict2)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict2[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict2

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm(completed).csv', usecols=[3], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70) #THIS NEEDS CHANGING, IT'S ONLY TRAINING THE LAST 30% OF START#
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict4 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict4 = scaler.inverse_transform(testPredict4)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict4[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict4

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm(completed).csv', usecols=[4], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70) #THIS NEEDS CHANGING, IT'S ONLY TRAINING THE LAST 30% OF START#
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict5 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict5 = scaler.inverse_transform(testPredict5)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict5[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict5

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
preDur = np.c_[testPredict]
prePit = np.c_[testPredict2]
preSta = np.c_[testPredict3]
preEnd = np.c_[testPredict4]
preSte = np.c_[testPredict5]

np.savetxt('ltsm (predicted complete).csv',np.hstack([preDur,prePit,(preSta-preSta[0]),(preEnd-preEnd[0]),preSte]), delimiter=',', header = 'duration,pitch,start,end,step', comments='', fmt='%1.2f')

In [ ]:
# BELOW IS EACH CELL FOR TRAINING THE DATASET IN INDIVIDUAL PIECES, BETTER FOR PC'S WITH LESS RAM#
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm(completed).csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
preDur = np.c_[testPredict]
np.savetxt('ltsm (predicted duration).csv', np.c_[testPredict], delimiter=',', header = 'Predicted Duration', comments='', fmt='%1.2f')
# SAVE np.c_[testPredict] AS DIFFERENT VARIABLE EACH TIME#

In [ ]:

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm(completed).csv', usecols=[0], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict2 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict2 = scaler.inverse_transform(testPredict2)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict2[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict2

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
prePit = np.c_[testPredict2]
np.savetxt('ltsm (predicted pitch).csv', np.c_[testPredict2], delimiter=',', header = 'Predicted Duration', comments='', fmt='%1.2f')

In [ ]:

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm(completed).csv', usecols=[2], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70) #THIS NEEDS CHANGING, IT'S ONLY TRAINING THE LAST 30% OF START#
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict3 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict3 = scaler.inverse_transform(testPredict3)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict3[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict3

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
preSta = np.c_[testPredict3]
np.savetxt('ltsm (predicted start).csv',np.c_[testPredict3], delimiter=',', header = 'Predicted Start', comments='', fmt='%1.2f')

In [ ]:

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm(completed).csv', usecols=[3], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70) #THIS NEEDS CHANGING, IT'S ONLY TRAINING THE LAST 30% OF START#
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict4 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict4 = scaler.inverse_transform(testPredict4)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict4[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict4

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
preEnd = np.c_[testPredict4]
np.savetxt('ltsm (predicted end).csv',np.c_[testPredict4], delimiter=',', header = 'Predicted End', comments='', fmt='%1.2f')

In [ ]:

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#LOAD THE DATA FOR LEARNING WITH#
dataframe = pandas.read_csv('ltsm(completed).csv', usecols=[4], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.figure(figsize = (15,10))
plt.plot(dataset)
plt.show()
numpy.random.seed(seed) #NEED TO FIX THE RANDOM NUMBER, MAKES THE RESULTS REPRODUCIBLE#

#NORMALISE THE DATA FROM 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#TRAIN 70% OF THE DATA, LEAVING 30 FOR TESTING
train_size = int(len(dataset) * 0.70) #THIS NEEDS CHANGING, IT'S ONLY TRAINING THE LAST 30% OF START#
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Amount of data for training:',len(train),',', 'Amount of data for testing:',len(test))

#DATASET CREATION, LOOK_BACK VALUE DEFINES HOW FAR AHEAD IT PREDICTS, Eg// X = DURATION AND GIVEN PITCH AND Y IS DURATION AT PITCH + LOOK_BACK
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

#RESHAPE THE DATAFRAME
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#SHAPE TO: SAMPLES, TIME STEP, FEATURES
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#CREATE AND FIT THE LTSM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2) 

#PREDICT
trainPredict = model.predict(trainX)
testPredict5 = model.predict(testX)

#INVERT
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict5 = scaler.inverse_transform(testPredict5)
testY = scaler.inverse_transform([testY])

#RMS ERROR
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict5[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#GET PREDICTIONS READY FOR PLOTTING
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict5

#PLOT THE BASE DATA ALONG WITH PREDICTED DATA
plt.figure(figsize = (20, 6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
preSte = np.c_[testPredict5]
np.savetxt('ltsm (predicted step).csv',np.c_[testPredict5], delimiter=',', header = 'Predicted Step', comments='', fmt='%1.2f')

In [ ]:
#COMBINE THE TRAINED FILES INTO ONE CSV
import os
import glob
import pandas as pd
df["duration"] = pd. read_csv("ltsm (predicted duration).csv")
df.to_csv("combined.csv", index=False)
df["pitch"] = pd. read_csv("ltsm (predicted pitch).csv")
df.to_csv("combined.csv", index=False)
df["start"] = pd. read_csv("ltsm (predicted start).csv")
df.to_csv("combined.csv", index=False)
df["end"] = pd. read_csv("ltsm (predicted end).csv")
df.to_csv("combined.csv", index=False)
df["step"] = pd. read_csv("ltsm (predicted step).csv")
df.to_csv("combined.csv", index=False)

In [ ]:
#CREATE THE FINAL MIDI FILE, ONLY USING THE FIRST 20,000 ENTRIES TO SAVE TIME AND CUT DOWN THE FILE LENGTH
DataFrame = pd.read_csv('combined.csv', engine='python', nrows = 20000)
def notes_to_midi(
  notes: DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm
print(DataFrame)
example_file = 'final.midi'
example_pm = notes_to_midi(
    DataFrame, out_file=example_file, instrument_name=instrument_name)
display_audio(example_pm)

In [ ]:
plot_piano_roll(DataFrame, count=200)
plot_piano_roll(DataFrame)

In [ ]:
DataFrame = pd.read_csv('combined.csv', engine='python')
def notes_to_midi(
  notes: DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm
print(DataFrame)
example_file = 'final full.midi'
example_pm = notes_to_midi(
    DataFrame, out_file=example_file, instrument_name=instrument_name)
display_audio(example_pm)